

 * add bias potential
 * perform MC sampling, particle in potential with adaptive force scaling
   * keep track of added bias potential at each position

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import random
import math
import seaborn as sns

# generate reasonable potential landscape

* DFT random numbers, filter with Gaussian

In [ ]:
length = 500
randomF = np.random.rand(length) * signal.gaussian(2*length, std=length/10)[length:]
completeFourier = np.concatenate([randomF])
energy = np.abs(np.square(np.fft.rfft(completeFourier)[length//10:]))

# MC sampling in landscape

In [ ]:
def new_position(energy, position, bias):
    direction = 2 * random.randint(0, 1) - 1

    # reflective walls
    if (position+direction < 0) or (position+direction >= len(energy)):
        direction = -direction

    energyChange = energy[position+direction] - energy[position] + direction * bias
    if energyChange < 0:
        return position + direction
    if random.random() < math.exp(-energyChange):
        return position + direction
    else:
        return position
    
def MCSamplingWithAdaptiveForceScaling(n_steps, energy):
    bias = -0.1
    
    timeconstant = 10
    inverseTimeConstant = 1/timeconstant
    
    weightedSum = 0.
    weightedCount = 0.
    
    position = 0
    result = [math.nan] * n_steps
    
    biases = [math.nan] * n_steps
    biasSum = [0] * len(energy)
    count = [0] * len(energy)
    
    for step in range(n_steps):
        
        position = new_position(energy, position, bias)
        result[step] = position
        
        weightedSum  = position + (1 - inverseTimeConstant) * weightedSum
        weightedCount = 1 + (1 - inverseTimeConstant) * weightedCount
        biases[step] = bias
        biasSum[position] += bias
        count[position] += 1
        if (position * weightedCount > weightedSum):
            bias /= 1. + inverseTimeConstant
        else:
            bias *= 1. + 2 * inverseTimeConstant
        
        if (position==len(energy)-1):
            break
            
    averageBias = [b/c if c>0 else 0 for b,c in zip(biasSum, count) ]
    return result[:step], biases[:step], averageBias, step


def MCSamplingWithBias(n_steps, energy, bias):

    position = 0
    result = [math.nan] * n_steps
    
    for step in range(n_steps):
        
        position = new_position(energy, position, bias)
        result[step] = position
        
        if (position==len(energy)-1):
            break
    
    return result[:step], step

In [ ]:
repeats = 1000
mfpt = [0] * repeats
averageBias = [None] * repeats
for repeat in range(repeats):
    positions,biases, averageBias[repeat], mfpt[repeat] = MCSamplingWithAdaptiveForceScaling(10000,energy)
meanAdaptive = np.mean(mfpt)
stdAdaptive = np.std(mfpt)
print(meanAdaptive)

In [ ]:
repeats = 1000
mfptFixedForce = [0] * repeats
for repeat in range(repeats):
    positions, mfptFixedForce[repeat] = MCSamplingWithBias(10000,energy,-1.675)
meanFixed = np.mean(mfptFixedForce)
stdFixed = np.std(mfptFixedForce)
print(meanFixed)

In [ ]:
sns.set_context('poster')
sns.set_style('ticks')
fig,ax = plt.subplots(figsize=(20,10))
ax.plot(energy,'k')

metaBias = np.mean(averageBias, axis=0)
pmfWithAdaptiveForce = energy+np.cumsum(metaBias)
ax.plot(pmfWithAdaptiveForce,color = '#991111')

constantBias = np.arange(0,len(energy)) * -1.675
ax.plot(energy+constantBias,color = '#222299')

fig.savefig('pmfwithbias.svg')
sns.despine(offset=10)

# Animation

In [ ]:
%matplotlib

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation

# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()
ax.plot(energy)

line, = ax.plot([], [], '.', ms=20)

# animation function.  This is called sequentially
def animate(i):
    x = positions[i]
    y=energy[x]
    line.set_data(x, y)
    return line,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, frames=len(positions), interval=20)

# save the animation as an mp4.  This requires ffmpeg or mencoder to be
# installed.  The extra_args ensure that the x264 codec is used, so that
# the video can be embedded in html5.  You may need to adjust this for
# your system: for more information, see
# http://matplotlib.sourceforge.net/api/animation_api.html
# anim.save('basic_animation.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
anim.save('animation.gif', writer='imagemagick', fps=60)
plt.show()